Import packages, and dataset



In [10]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

# Set your data directory
# ...existing code...
data_dir = r"D:\\DNA Folder\\DNA Project\\dataset\\data"
# ...existing code... # Change this path
categories = ["with_mask", "without_mask"]
img_size = 224
images = []
labels = []

# Load and preprocess data
print("[INFO] Loading images...")
for category in categories:
    path = os.path.join(data_dir, category)
    if not os.path.exists(path):
        print(f"[WARNING] Directory does not exist: {path}. Skipping this category.")
        continue
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(img_size, img_size))
        image = img_to_array(image)
        images.append(image)
        labels.append(category)

# Convert labels to numerical format
labels = [0 if label == "with_mask" else 1 for label in labels]
labels = to_categorical(labels)

if len(images) == 0:
    raise ValueError("[ERROR] No images were loaded. Please check your dataset path and folder structure.")

images = np.array(images, dtype="float32")
images = images / 255.0

# Split data into training and testing sets
(trainX, testX, trainY, testY) = train_test_split(images, labels, test_size=0.20, stratify=labels, random_state=42)

# Create a data augmenter for real-time data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

[INFO] Loading images...


d:\DNA Folder\DNA Project\.venv\Lib\site-packages\PIL\Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Building the model

In [11]:
print("[INFO] Building and compiling the model...")

# Load the MobileNetV2 network, freezing its base layers
base_model = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Add new layers on top for classification
head_model = base_model.output
head_model = AveragePooling2D(pool_size=(7, 7))(head_model)
head_model = Flatten(name="flatten")(head_model)
head_model = Dense(128, activation="relu")(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense(2, activation="softmax")(head_model)

# Combine the base model and new layers
model = Model(inputs=base_model.input, outputs=head_model)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
optimizer = Adam(learning_rate=0.0001)
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

[INFO] Building and compiling the model...


C:\Users\saiud\AppData\Local\Temp\ipykernel_19480\4094826070.py:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 6s 1us/step


Training the model

In [12]:
epochs = 20
batch_size = 32

print("[INFO] Training the model...")
model.fit(
    aug.flow(trainX, trainY, batch_size=batch_size),
    steps_per_epoch=len(trainX) // batch_size,
    validation_data=(testX, testY),
    validation_steps=len(testX) // batch_size,
    epochs=epochs)

# Save the trained model to disk
print("[INFO] Saving the model...")
model.save("mask_detector.h5")

[INFO] Training the model...


d:\DNA Folder\DNA Project\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 193s 984ms/step - accuracy: 0.8923 - loss: 0.2967 - val_accuracy: 0.9689 - val_loss: 0.1097
Epoch 2/20


d:\DNA Folder\DNA Project\.venv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


188/188 ━━━━━━━━━━━━━━━━━━━━ 29s 151ms/step - accuracy: 1.0000 - loss: 0.1100 - val_accuracy: 0.9689 - val_loss: 0.1094
Epoch 3/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 200s 1s/step - accuracy: 0.9724 - loss: 0.1046 - val_accuracy: 0.9702 - val_loss: 0.0827
Epoch 4/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 30s 155ms/step - accuracy: 0.9688 - loss: 0.0897 - val_accuracy: 0.9709 - val_loss: 0.0825
Epoch 5/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 179s 952ms/step - accuracy: 0.9739 - loss: 0.0860 - val_accuracy: 0.9762 - val_loss: 0.0670
Epoch 6/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 28s 146ms/step - accuracy: 1.0000 - loss: 0.0670 - val_accuracy: 0.9755 - val_loss: 0.0669
Epoch 7/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 167s 890ms/step - accuracy: 0.9764 - loss: 0.0732 - val_accuracy: 0.9788 - val_loss: 0.0616
Epoch 8/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 28s 144ms/step - accuracy: 0.9688 - loss: 0.0768 - val_accuracy: 0.9795 - val_loss: 0.0616
Epoch 9/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 168s 893ms/step - accuracy: 0.9809 - loss: 0.0583 - va